# Thomas Quinlan - Final Year Project 

## BSc Mathematical Sciences (Single Honours)


The following notebook contains all code that was written to produce the figures and results obtained in my final year project entitled "The Risk of Extinction in Predator-Prey Systems Subject to Chaotic Climate Variability." 

Disclaimers: 

* The only code that is not included within this notebook is that which was used in XPPAUT to produce the bifurcation diagrams seen in section 4.2 of the report. The code for this was minimal and the analysis was mostly conducted through the XPPAUT user interface. 
* Reproduction of the bifurcation diagrams here will not be possible without the data file exported from XPPAUT. 
* Some figures will not look identical to those that appear in the written report, as certain details were added to the figures in Inkscape after they were produced here. Any details added in inkscape were purely for visual aesthetics and they do not impact on the quantitative results of the work.  
* At times I use the python variable name r to refer to the parameter $\rho$. I apologise for any confusion this causes. 
* I do not claim that all code written in this notebook is original work. I drew upon my lecture notes from modules I have taken as well as online sources and python documentation. 


### Section 2: Types of Population Growth Models

#### Figure 1:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

In [ ]:
def rk4(f, h, a, b, alpha):
   
    N = int((b-a)/h)
    y_t = np.zeros(N+1)
    T = np.zeros(N+1)
    w = alpha
    y_t[0] = w
    t = a
    T[0] = a
    
    for i in range(1,N+1):
        
        k1 = h*f(w)
        k2 = h*f(w+k1/2)
        k3 = h*f(w+k2/2)
        k4 = h*f(w+k3)
        
        w = w + (k1+2*k2+2*k3+k4)/6
        y_t[i] = w
        t = t + h
        T[i] = t
    
    return y_t, T

In [ ]:
def linear(U_0,r,t):
    return U_0*(np.exp(r*t))

def Logistic(U_0,r,mu,t):
    C = (r-mu*U_0)/(U_0)
    return r/(mu+C*(np.exp(-r*t)))

def Allee(U_0,b,l,g,A,B,h):
    f = lambda u: -b*u + l*u**2 - g*u**3
    sol = rk4(f, h, A, B, U_0)
    return sol

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(15,4))

# Linear Model
myt = np.linspace(0,10,100)
U_0 = 1
r = 0.22
axs[0].plot(myt,linear(U_0,r,myt),'b',linewidth=2,label=r'$N(0)=1$')
axs[0].text(0.25,10.3,'(a)',fontsize='large')
axs[0].set(xlim=(0,10),ylim=(0,10))
axs[0].set_title('The Linear Model')
axs[0].legend(loc='lower right')

# Logistic Model
myt = np.linspace(0,25,100)
U_1 = 1 
U_2 = 12
r = 0.22
mu = 0.22/8
axs[1].plot(myt,Logistic(U_1,r,mu,myt),'b',linewidth=2,label=r'$N_{1}(0)=1$')
axs[1].plot(myt,Logistic(U_2,r,mu,myt),'r',linewidth=2,label=r'$N_{2}(0)=12$')
axs[1].text(0.2,7.275,r'$e_{1}=K$',color='k',fontsize='large')
axs[1].text(0.25,13.3,'(b)',fontsize='large')
axs[1].hlines(8,0,20,'k','dashed')
axs[1].set(xlim=(0,25),ylim=(0,13))
axs[1].set_title('The Logistic Model')
axs[1].legend(loc='lower right')

# Allee Model
U_1 = 2
U_2 = 5
U_3 = 10
b = 0.22
l = 0.1 
g = 0.009
A = 0
B = 20
h = 0.01
(u1_t,T) = Allee(U_1,b,l,g,A,B,h)
(u2_t,T) = Allee(U_2,b,l,g,A,B,h)
(u3_t,T) = Allee(U_3,b,l,g,A,B,h)
axs[2].plot(T,u1_t,'b',linewidth=2,label=r'$N_{1}(0)=2$')
axs[2].plot(T,u2_t,color='orange',linewidth=2,label=r'$N_{2}(0)=5$')
axs[2].plot(T,u3_t,'r',linewidth=2,label=r'$N_{3}(0)=10$')
axs[2].text(0.25,11.3,'(c)',fontsize='large')
axs[2].text(0.2,7.4,r'$e_{1}=K$',color='k',fontsize='large')
axs[2].text(0.2,3.3,r'$e_{2}=A$',color='k',fontsize='large')
axs[2].set_title('The Allee Model')
axs[2].hlines(8.1,0,20,'k','dashed')
axs[2].hlines(3.02,0,20,'k','dashed')
axs[2].set(xlim=(0,20),ylim=(0,11))
axs[2].legend(loc='center right')

for ax in axs.flat:
    ax.set(xlabel=r'$t$', ylabel=r'$N(t)$')
    plt.rc('axes', labelsize=13) 
    
plt.savefig('Pop Growth All In One.pdf',format='pdf')

### Section 3: Predator-Prey Models and Functional Response


#### Figure 2: 

In [ ]:
# Type 1 
def rcons_1(a,N):
    return a*N

# Type 2
def rcons_2(a,h,N):
    return (a*N)/(1+h*a*N)

# Type 3
def rcons_3(a,h,N):
    return (a*N**2)/(1+h*a*N**2)

# Non Monotonic
def nm(a,h,b,N):
    return ((a*N**2)/(1+h*a*N**2))*np.exp(-b*N)

# N values for most plots
N = np.linspace(0,100,1000)

# N values for cut-off type 1 plot
Ns = 50
Ns_list = np.linspace(0,Ns,1000)

# Mu values for varying the type 3 plot
ms = [2.5,3.5]
def rcons_3_m(a,h,m,N):
    return (a*N**m)/(1+h*a*N**m)

In [ ]:
# Examining the derivative of the non monotonic plot
def df_nm(a,h,b,N):
    return (a*(-a*b*np.exp(-b*N)*N**(4)*h-b*np.exp(-b*N)*N**(2) +  
               2*np.exp(-b*N)*N))/(1+a*N**(2)*h)**(2)

plt.plot(N,df_nm(0.004,0.2,0.019,Ns_list))

plt.hlines(0,-2,100)

In [ ]:
# Finding the Maximum Point of the non monotonic plot

from scipy.optimize import fsolve

a = 0.004
h = 0.2
b = 0.019

f = lambda N: (a*(-a*b*np.exp(-b*N)*N**(4)*h-b*np.exp(-b*N)*N**(2) + 
                  2*np.exp(-b*N)*N))/(1+a*N**(2)*h)**(2)

xroot = fsolve(f,[42])[0]
print(xroot)
f_xroot = nm(a,h,b,xroot) 
print(f_xroot)


In [ ]:
fig, axs = plt.subplots(2,2,figsize=(10,7))

# Type 1
a = 0.02
axs[0,0].plot(N,rcons_1(a,N),linewidth=2,color='b',label=r'$F_{1}(N)$')
axs[0,0].plot(Ns_list,rcons_1(a,Ns_list),color='k',linestyle='dashed')
axs[0,0].hlines(rcons_1(a,Ns),Ns,100,'k','dashed')
axs[0,0].text(65,rcons_1(a,Ns)+0.07,r'$F_{1}^*(N)= \alpha N^{*}$',
              color='black',fontsize='large')
axs[0,0].text(5,2.05,'(a)',color='black',fontsize='large')
axs[0,0].legend(loc = 'lower right')
axs[0,0].set_title('Type I')
axs[0,0].set(xlim=(0,100),ylim=(0,2))

# Type 2
a = 0.1
h = 0.625
axs[0,1].plot(N,rcons_2(a,h,N),linewidth=2,color='b',label=r'$F_{2}(N)$')
axs[0,1].text(5,2.05,'(b)',color='black',fontsize='large')
axs[0,1].hlines(1/h,0,100,'k','dashed')
axs[0,1].set(xlim=(0,100),ylim=(0,2))
axs[0,1].set_title('Type II')
axs[0,1].legend(loc='lower right')

# Type 3
a = 0.004
h = 0.625
axs[1,0].plot(N,rcons_3(a,h,N),linewidth=2,color='b',label=r'$F_{3}(N)$')
axs[1,0].hlines(1/h,0,1000,'k','dashed')
axs[1,0].text(5,2.05,'(c)',color='black',fontsize='large')
ms = [2.5,3.5]
for m in ms:
    axs[1,0].plot(N,rcons_3_m(a,h,m,N),linewidth=2,color='b')  
axs[1,0].text(21.5,1.25,r'$\mu_{1}$',color='k',fontsize='large')
axs[1,0].text(13,1.43,r'$\mu_{2}$',color='k',fontsize='large')
axs[1,0].text(26.5,0.9,r'$\mu_{0}$',color='k',fontsize='large')
axs[1,0].set(xlim=(0,100),ylim=(0,2))
axs[1,0].set_title('Type III')
axs[1,0].legend(loc = 'lower right')

# Type 4
a = 0.004
h = 0.2
b = 0.019
axs[1,1].plot(N,nm(a,h,b,N),linewidth=2,color='b',label=r'$F_{4}(N)$')
axs[1,1].plot(xroot, f_xroot, 'ro',label=r'$F_{4}(N^*)=M$')
axs[1,1].text(5,2.05,'(d)',color='black',fontsize='large')
axs[1,1].vlines(xroot,0,f_xroot,'k',linestyle='dashed')
axs[1,1].hlines(f_xroot,0,xroot,'k',linestyle='dashed')
axs[1,1].set(xlim=(0,100),ylim=(0,2))
axs[1,1].set_title('Non-Monotonic')
axs[1,1].legend(loc = 'lower right')

for ax in axs.flat:
    ax.set(xlabel='Prey Density', ylabel='Prey Consumption Rate')
    plt.rc('axes', labelsize=12) 

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

plt.savefig('FR all in one.pdf',format='pdf')

### Section 4: The May Model

#### Figure 3:

In [ ]:
c = 0.22
a = 505
b = 0.3
s = 0.85
q = 205
m = 0.03
e = 0.031

R = 1.3

In [ ]:
def New_Allee(N,r,v):
    return (r*N) * ( 1 - (c/r)*N) * ((N-m)/(v+N))

In [ ]:
v_vals = [0.003,0.09]
colors = ['blue','orange']

Ns = np.linspace(0,15,10000)

fig, (ax1, ax2) = plt.subplots(1,2,figsize=(12,4))



for i, v in enumerate(v_vals):
    ax1.plot(Ns,New_Allee(Ns,R,v),label=r'$\nu_{%.0f}=%.3f$'%(i+1,v),
             lw=2,color=colors[i])
    
root_1 = R/c
root_2 = m
root_3 = 0

ax1.plot(root_1,0,'ro')
ax1.plot(root_2,0,'ro')
ax1.plot(root_3,0,'ro')

ax1.hlines(0,0,15,'k')
ax1.vlines(0,-10,10,'k')

ax1.set_xlim(root_3,root_2)
ax1.set_ylim(-0.03,0.03)

ax1.set_ylabel(r'$\dot{N}$',fontsize=14,rotation=0,labelpad=15)
ax1.set_xlabel(r'$N$',fontsize=14)

ax1.legend()



for i, v in enumerate(v_vals):
    ax2.plot(Ns,New_Allee(Ns,R,v),label=r'$\nu_{%.0f}=%.3f$'%(i+1,v),
             lw=2.2,color=colors[i])
    
ax2.plot(root_1,0,'ro')
ax2.plot(root_2,0,'ro')
ax2.plot(root_3,0,'ro')

ax2.hlines(0,0,15,'k')
ax2.vlines(0,-10,10,'k')
ax2.set_xlim(root_2,6.4)
ax2.set_ylim(-0.1,2)

ax2.set_xlabel(r'$N$',fontsize=14)

ax2.legend()



plt.savefig('NuEffect.pdf',format='pdf')

In [ ]:
c = 0.22
a = 505
b = 0.3
s = 0.85
q = 205
m = 0.03
v = 0.003
e = 0.031

In [ ]:
def cubic(N,r):
    return N**3-(m - b + (r/c) - (a/(c*q)))*N**2-(b*m + (r*(b-m))/c - 
                      (a*(v+e))/(c*q))*N+((r*b*m)/(c) + (a*v*e)/(c*q))

In [ ]:
# rho_1:

r = 0.2

f = lambda N: N**3-(m - b + (r/c) - (a/(c*q)))*N**2-(b*m + (r*(b-m))/c -
                     (a*(v+e))/(c*q))*N+((r*b*m)/(c) + (a*v*e)/(c*q))

root1_1 = fsolve(f,[-12])[0]
print(root1_1)

# rho_2:

r = 2

f = lambda N: N**3-(m - b + (r/c) - (a/(c*q)))*N**2-(b*m + (r*(b-m))/c - 
                     (a*(v+e))/(c*q))*N+((r*b*m)/(c) + (a*v*e)/(c*q))

root2_1 = fsolve(f,[0])[0]
root2_2 = fsolve(f,[1])[0]
root2_3 = fsolve(f,[-4])[0]

print(root2_1,root2_2,root2_3)

# rho_3:

r = 3.3

f = lambda N: N**3-(m - b + (r/c) - (a/(c*q)))*N**2-(b*m + (r*(b-m))/c - 
                     (a*(v+e))/(c*q))*N+((r*b*m)/(c) + (a*v*e)/(c*q))

root3_1 = fsolve(f,[-1])[0]
root3_2 = fsolve(f,[0])[0]
root3_3 = fsolve(f,[4])[0]

print(root3_1,root3_2,root3_3)



In [ ]:
r_vals = [0.2,2,3.3]
N_vals = np.linspace(-20,20,100000)
cols = ['blue','orange','red']



plt.figure(figsize=(10,6))



for i, r in enumerate(r_vals):
    col = cols[i]
    plt.plot(N_vals,cubic(N_vals,r),lw=2,color=col,
             label=r'$\rho_{%i}=%.1f$'%(i+1,r))

plt.plot(0,0,'go',markersize=4)

plt.plot(root1_1,0,'go',markersize=4, label='roots')

plt.plot(root2_1,0,'go',markersize=4)
plt.plot(root2_2,0,'go',markersize=4)
plt.plot(root2_3,0,'go',markersize=4)

plt.plot(root3_1,0,'go',markersize=4)
plt.plot(root3_2,0,'go',markersize=4)
plt.plot(root3_3,0,'go',markersize=4)

plt.hlines(0,-20,20,'k')
plt.vlines(0,-20,20,'k')
plt.ylim(-20,20)
plt.xlim(-12,12)
plt.xlabel('N',fontsize=14)
plt.ylabel(r'$\mathcal{P}$(N)',fontsize=14,rotation='horizontal',
           labelpad=5)
plt.legend()



N_super_zoom = np.linspace(-0.1,0.1,10000)
P_N_super_zoom_1 = cubic(N_super_zoom,r)
axes = plt.axes([.24, .67, .2, .15])
for i, r in enumerate(r_vals):
    col = cols[i]
    axes.plot(N_super_zoom,cubic(N_super_zoom,r),lw=1.5,color=col)
    
plt.plot(root1_1,0,'go',markersize=4)

plt.plot(root2_1,0,'go',markersize=4)
plt.plot(root2_2,0,'go',markersize=4)
plt.plot(root2_3,0,'go',markersize=4)

plt.plot(root3_1,0,'go',markersize=4)
plt.plot(root3_2,0,'go',markersize=4)
plt.plot(root3_3,0,'go',markersize=4)
    
plt.ylim(-0.05,0.05)
plt.xlim(-0.1,0.1)
plt.hlines(0,-0.1,0.1,'k',lw=0.8)
plt.vlines(0,-0.1,0.1,'k',lw=0.8)




plt.savefig('P(N) roots.pdf',format='pdf')

#### Figure 5: 

In [ ]:
c = 0.22
a = 505
b = 0.3
s = 0.85
q = 205
m = 0.03
v = 0.003
e = 0.031

In [ ]:
A = m - b + r/c - a/(c*q)
B = b*m + (r*(b-m))/c - (a*(v+e))/(c*q)
C = (r*b*m)/(c) + (a*v*e)/(c*q) 

In [ ]:
# This cell allows me to check which expressions correspond to which roots

r=4

N1 = ((2*(A**3) + 3*((3)**(1/2))*((-4*(A**3)*C-(A**2)*(B**2)-18*A*B*C - 
           4*(B**3)+27*(C**2))**(1/2)) + 9*A*B - 27*C)**(1/3))/(3*((2)**(1/3))) - \
            (((2)**(1/3))*(-((A)**2)-3*B))/((3)*((2*(A**3) + 
          3*((3)**(1/2))*((-4*(A**3)*C-(A**2)*(B**2)-18*A*B*C-4*(B**3)+27*(C**2))**(1/2))
          + 9*A*B - 27*C)**(1/3))) + (A)/(3)

N2 = (-(1-(1j)*((3)**(1/2)))*(2*(A**3) + 3*((3)**(1/2))*((-4*(A**3)*C-(A**2)*(B**2) - 
          18*A*B*C-4*(B**3)+27*(C**2))**(1/2)) + 9*A*B - 27*C)**(1/3))/(6*((2)**(1/3))) + \
            ((1+(1j)*((3)**(1/2)))*(-((A)**2)-3*B))/((3*(((2)**(1/3))**2))*((2*(A**3) + 
         3*((3)**(1/2))*((-4*(A**3)*C-(A**2)*(B**2)-18*A*B*C-4*(B**3)+27*(C**2))**(1/2)) + 
         9*A*B - 27*C)**(1/3))) + (A)/(3)

N3 = (-(1+(1j)*((3)**(1/2)))*(2*(A**3) + 3*((3)**(1/2))*((-4*(A**3)*C-(A**2)*(B**2) - 
          18*A*B*C-4*(B**3)+27*(C**2))**(1/2)) + 9*A*B - 27*C)**(1/3))/(6*((2)**(1/3))) + \
            ((1-(1j)*((3)**(1/2)))*(-((A)**2)-3*B))/((3*(((2)**(1/3))**2))*((2*(A**3) + 
         3*((3)**(1/2))*((-4*(A**3)*C-(A**2)*(B**2)-18*A*B*C-4*(B**3)+27*(C**2))**(1/2)) + 
         9*A*B - 27*C)**(1/3))) + (A)/(3)

print(N1)
print(N2)
print(N3)

In [ ]:
def N4(r):
    
    A = m - b + r/c - a/(c*q)
    B = b*m + (r*(b-m))/c - (a*(v+e))/(c*q)
    C = (r*b*m)/(c) + (a*v*e)/(c*q) 
    
    N_4 = (-(1+(1j)*((3)**(1/2)))*(2*(A**3) + 3*((3)**(1/2)) * 
               ((-4*(A**3)*C-(A**2)*(B**2)-18*A*B*C-4*(B**3) + 
                 27*(C**2))**(1/2)) + 9*A*B - 27*C)**(1/3)) / \
                (6*((2)**(1/3))) + ((1-(1j)*((3)**(1/2))) * 
                (-((A)**2)-3*B))/((3*(((2)**(1/3))**2))*((2*(A**3) + 
              3*((3)**(1/2))*((-4*(A**3)*C-(A**2)*(B**2) - 
               18*A*B*C-4*(B**3)+27*(C**2))**(1/2)) + 9*A*B - 
              27*C)**(1/3))) + (A)/(3)
    
    return N_4.real

def N5(r):
    
    A = m - b + r/c - a/(c*q)
    B = b*m + (r*(b-m))/c - (a*(v+e))/(c*q)
    C = (r*b*m)/(c) + (a*v*e)/(c*q) 
    
    N_5 = ((2*(A**3) + 3*((3)**(1/2))*((-4*(A**3)*C-(A**2) * 
                (B**2)-18*A*B*C-4*(B**3)+27*(C**2))**(1/2)) + 
                9*A*B - 27*C)**(1/3))/(3*((2)**(1/3))) - \
                (((2)**(1/3))*(-((A)**2)-3*B))/((3)*((2*(A**3) + 
              3*((3)**(1/2))*((-4*(A**3)*C-(A**2)*(B**2) - 
               18*A*B*C-4*(B**3)+27*(C**2))**(1/2)) + 9*A*B - 
              27*C)**(1/3))) + (A)/(3)
    
    return N_5.real

def N6(r):
    
    A = m - b + r/c - a/(c*q)
    B = b*m + (r*(b-m))/c - (a*(v+e))/(c*q)
    C = (r*b*m)/(c) + (a*v*e)/(c*q) 
    
    N_6 = (-(1-(1j)*((3)**(1/2)))*(2*(A**3) + 3*((3)**(1/2)) * 
               ((-4*(A**3)*C-(A**2)*(B**2)-18*A*B*C-4*(B**3) + 
                 27*(C**2))**(1/2)) + 9*A*B - 27*C)**(1/3)) / \
                (6*((2)**(1/3))) + ((1+(1j)*((3)**(1/2))) * 
                (-((A)**2)-3*B))/((3*(((2)**(1/3))**2)) * 
              ((2*(A**3) + 3*((3)**(1/2))*((-4*(A**3)*C - 
              (A**2)*(B**2)-18*A*B*C-4*(B**3)+27*(C**2))**(1/2)) 
                + 9*A*B - 27*C)**(1/3))) + (A)/(3)
    
    return N_6.real


The value $\rho=1.20324$ for the location of the fold point F was obtained from the analysis carried out in XPPAUT. 

In [ ]:
r_vals1 = np.linspace(1.20324,4,1000)
r_vals2 = np.linspace(0,4,1000)

N4vec = np.vectorize(N4)
N4_r = N4vec(r_vals1)

N5vec = np.vectorize(N5)
N5_r = N5vec(r_vals1)

N6vec = np.vectorize(N6)
N6_r = N6vec(r_vals2)


fig, (ax1, ax2) = plt.subplots(1,2,figsize=(12,4))

ax1.plot(r_vals1,N4_r,color='orange',lw=2,label=r'$N_4(\rho)$')
ax1.plot(r_vals1,N5_r,color='blue',lw=2,label=r'$N_5(\rho)$')
ax1.plot(r_vals2,N6_r,color='red',lw=2,label=r'$N_6(\rho)$')

ax1.plot(1.20324,N4vec(1.20324),'go',markersize=6)

ax1.set_xlim(-0.2,4)
ax1.set_ylim(-7,7)

ax1.set_xlabel(r'$\rho$',fontsize=14)
ax1.set_ylabel('N',fontsize=14,rotation=0)

ax1.hlines(0,-0.20,4)
ax1.vlines(0,-7,7)

ax1.text(3.75,7.2,r'$(a)$',color='black', fontsize=12)
ax1.text(1.1,N4vec(1.20324)+0.3,'F',color='black',fontsize=12)

ax1.legend(loc='lower right')


ax2.plot(r_vals1,N4_r,color='orange',lw=2)
ax2.plot(r_vals1,N5_r,color='blue',lw=2)

ax2.plot(1.20324,N4vec(1.20324),'go',markersize=6)

ax2.set_xlim(-0.2,4)
ax2.set_ylim(10**(-2),10)

ax2.hlines(0,-0.2,4)
ax2.vlines(0,0,10)

ax2.set_yscale('log')

ax2.set_xlabel(r'$\rho$',fontsize=14)

ax2.text(3.75,10.8,r'$(b)$',color='black', fontsize=12)
ax2.text(1.02,N4vec(1.20324)-0.01,'F',color='black',fontsize=12)

plt.savefig('Ns_vs_Rho.pdf',format='pdf')

#### Figure 6: 


In [ ]:
import pandas as pd

In [ ]:
# Reading in the output from XPPAUT

# We must extract the output for each section of the bifurcation plot,
# so that we can reconstruct it here.

dat = pd.read_excel(r'.\May_limitcycle.xlsx')
dat_array = pd.DataFrame.to_numpy(dat)

rho_vals = dat_array[:,0]
Max_vals = dat_array[:,1]
Min_vals = dat_array[:,2]




fold_dat = pd.read_excel(r'.\Fold Bif for N variable.xlsx')

fold_dat_array = pd.DataFrame.to_numpy(fold_dat)

stable2_N = fold_dat_array[0:52,1]
stable2_rho = fold_dat_array[0:52,0]

unstable_N = fold_dat_array[52:165,1]
unstable_rho = fold_dat_array[52:165,0]

stable1_N = fold_dat_array[165:185,1] #177
stable1_rho = fold_dat_array[165:185,0] #177

stablel_N = fold_dat_array[177:244,1]
stablel_rho = fold_dat_array[177:244,0]




Pdat = pd.read_excel(r'.\Pptsdata.xlsx')

Pdat_array = pd.DataFrame.to_numpy(Pdat)

Pstableu_rho = Pdat_array[0:51,0]
Pstableu_P = Pdat_array[0:51,1] 

Punstable_rho = Pdat_array[51:165,0]
Punstable_P = Pdat_array[51:165,1] 

Pstablel_rho = Pdat_array[164:177,0]
Pstablel_P = Pdat_array[164:177,1] 

Plower_rho = Pdat_array[175:244,0]
Plower_P = Pdat_array[175:244,1] 

Phopfu_rho = Pdat_array[245:476,0]
Phopfu_P = Pdat_array[245:476,1] 

Phopfl_rho = Pdat_array[245:476,0]
Phopfl_P = Pdat_array[245:476,2] 

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(12,8))


ax1.plot(rho_vals,Max_vals,color='green',lw=2)
ax1.plot(rho_vals,Min_vals,color='green',lw=2)

ax1.plot(stable2_rho,stable2_N,color='black',lw=2)

ax1.plot(unstable_rho,unstable_N,'k--',lw=2)

ax1.plot(stable1_rho,stable1_N,color='black',lw=2)

ax1.plot(stablel_rho,stablel_N,'k--',lw=2)

ax1.plot(1.62326,0.399868,'rx',markersize=8)

ax1.plot(3.45852,5.72264,'rx',markersize=8)

ax1.plot(1.11445,0.089452,'ro',markersize=6)

ax1.text(3.95,12.5,r'$(a)$',color='black',fontsize=12)

ax1.text(3.85,7,r'$e_5$',color='black',fontsize=12)
ax1.text(3.85,0.3,r'$e_4$',color='black',fontsize=12)
ax1.text(1.08,0.34,r'$F$',color='black',fontsize=12)
ax1.text(1.42,0.71,r'$H_1$',color='black',fontsize=12)
ax1.text(3.5,5,r'$H_2$',color='black',fontsize=12)

ax1.set_ylabel('N', fontsize=14, rotation=0,labelpad = 10)


ax2.plot(rho_vals,Max_vals,color='green',lw=2)
ax2.plot(rho_vals,Min_vals,color='green',lw=2)

ax2.plot(stable2_rho,stable2_N,color='black',lw=2)

ax2.plot(unstable_rho,unstable_N,'k--',lw=2)

ax2.plot(stable1_rho,stable1_N,color='black',lw=2)

ax2.plot(stablel_rho,stablel_N,'k--',lw=2)

ax2.plot(1.62326,0.399868,'rx',markersize=8)

ax2.plot(3.45852,5.72264,'rx',markersize=8)

ax2.plot(1.11445,0.089452,'ro',markersize=6)

ax2.text(3.95,17,r'$(b)$',color='black',fontsize=12)

ax2.text(3.85,5.5,r'$e_5$',color='black',fontsize=12)
ax2.text(3.85,0.04,r'$e_4$',color='black',fontsize=12)
ax2.text(1.18,0.08,r'$F$',color='black',fontsize=12)
ax2.text(1.67,0.3,r'$H_1$',color='black',fontsize=12)
ax2.text(3.5,4,r'$H_2$',color='black',fontsize=12)

ax2.set_yscale('log')


ax3.plot(Phopfu_rho,Phopfu_P,color='green',lw=2)
ax3.plot(Phopfl_rho,Phopfl_P,color='green',lw=2)

ax3.plot(Pstableu_rho,Pstableu_P,color='black',lw=2)

ax3.plot(Punstable_rho,Punstable_P,'k--',lw=2)

ax3.plot(Pstablel_rho,Pstablel_P,color='black',lw=2)

ax3.plot(Plower_rho,Plower_P,'k--',lw=2)

ax3.plot(1.62326,0.00195395,'rx',markersize=8)

ax3.plot(3.45852,0.0261484,'rx',markersize=8)

ax3.plot(1.11445,0.000542964,'ro',markersize=6)

ax3.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

ax3.text(3.99,3.93e-2,r'$(c)$',color='black',fontsize=12)

ax3.text(3.85,3.2e-2,r'$e_5$',color='black',fontsize=12)
ax3.text(3.85,0.1e-2,r'$e_4$',color='black',fontsize=12)
ax3.text(1.08,0.13e-2,r'$F$',color='black',fontsize=12)
ax3.text(1.42,0.3e-2,r'$H_1$',color='black',fontsize=12)
ax3.text(3.51,2.5e-2,r'$H_2$',color='black',fontsize=12)


ax4.plot(Phopfu_rho,Phopfu_P,color='green',lw=2)
ax4.plot(Phopfl_rho,Phopfl_P,color='green',lw=2)

ax4.plot(Pstableu_rho,Pstableu_P,color='black',lw=2)

ax4.plot(Punstable_rho,Punstable_P,'k--',lw=2)

ax4.plot(Pstablel_rho,Pstablel_P,color='black',lw=2)

ax4.plot(Plower_rho,Plower_P,'k--',lw=2)

ax4.plot(1.62326,0.00195395,'rx',markersize=8)

ax4.plot(3.45852,0.0261484,'rx',markersize=8)

ax4.plot(1.11445,0.000542964,'ro',markersize=6)

ax4.text(3.99,5.2e-2,r'$(d)$',color='black',fontsize=12)

ax4.text(3.85,2.6e-2,r'$e_5$',color='black',fontsize=12)
ax4.text(3.85,0.033e-2,r'$e_4$',color='black',fontsize=12)
ax4.text(1.17,0.05e-2,r'$F$',color='black',fontsize=12)
ax4.text(1.67,0.15e-2,r'$H_1$',color='black',fontsize=12)
ax4.text(3.5,1.9e-2,r'$H_2$',color='black',fontsize=12)

ax4.set_yscale('log')


ax4.set_xlabel(r'$\rho$', fontsize=14)
ax3.set_xlabel(r'$\rho$', fontsize=14)
ax3.set_ylabel('P', fontsize=14, rotation=0,labelpad = 10)

plt.savefig('Bifurcation_Diagrams.pdf',format='pdf')

#### Figure 7:

In [ ]:
r=3
c=0.22
alpha=505
beta=0.3
s=0.85
q=205
mu=0.03
v=0.003
e=0.031

A = mu - beta + (r/c) - (alpha/(c*q))
B = beta*mu + r*(beta-mu)/c - alpha*(v+e)/(c*q)
C = (r*beta*mu)/c + (alpha*v*e)/(c*q)

N0 = N = (2 *A**3 + 3 *np.sqrt(3)* np.sqrt(-4* A**3 *C - A**2 *B**2
               - 18 *A *B *C - 4 *B**3 + 27 *C**2 + 0j) + 9 *A *B - 
              27 *C)**(1/3)/(3 *2**(1/3)) - (2**(1/3) *(-(A)**2 - 3 *B)) / \
              (3 *(2* A**3 + 3 *np.sqrt(3) *np.sqrt(-4* A**3 *C - 
                A**2 *B**2 - 18 *A *B *C - 4* B**3 + 27 *C**2 + 0j) + 
               9* A *B - 27 *C)**(1/3)) + A/3
N1 = -((1 + np.sqrt(-1+0j)*np.sqrt(3))*(2*A**3 + 3*np.sqrt(3) * 
                np.sqrt(-4*A**3*C - A**2 * B**2 - 18 *A *B *C - 
                4 *B**3 + 27 *C**2 + 0j) + 9 *A *B - 27 *C)**(1/3)) / \
                (6 *2**(1/3)) + ((1 - np.sqrt(-1+0j) *np.sqrt(3)) * 
                (-A**2 - 3 *B))/(3 *2**(2/3) *(2 *A**3 + 3 *np.sqrt(3) * 
               np.sqrt(-4 *A**3 *C - A**2 *B**2 - 18 *A *B *C - 4 *B**3 + 
               27 *C**2 + 0j) + 9 *A *B - 27 *C)**(1/3)) + A/3
N2 = -((1 - np.sqrt(-1+0j)*np.sqrt(3))*(2*A**3 + 3*np.sqrt(3) * 
               np.sqrt(-4*A**3*C - A**2 * B**2 - 18 *A *B *C - 
               4 *B**3 + 27 *C**2 + 0j) + 9 *A *B - 27 *C)**(1/3)) / \
               (6 *2**(1/3)) + ((1 + np.sqrt(-1+0j) *np.sqrt(3)) * 
               (-A**2 - 3 *B))/(3 *2**(2/3) *(2 *A**3 + 3 *np.sqrt(3) * 
               np.sqrt(-4 *A**3 *C - A**2 *B**2 - 18 *A *B *C - 4 *B**3 + 
               27 *C**2 + 0j) + 9 *A *B - 27 *C)**(1/3)) + A/3

N0r = np.real(N0)
N1r = np.real(N1)
N2r = np.real(N2)

P0r = (N0r+e)/q 
P1r = (N1r + e)/q
P2r = (N2r+3)/q

print(N0r)
print(P0r)
print(N1r)
print(P1r)
print(N2r)
print(P2r)

In [ ]:
def rk4(f, h, a, b, init):
    
    N = int((b-a)/h)
    y_t = np.zeros((N+1,2))
    T = np.zeros(N+1)
    w = init
    y_t[0,:] = w
    t = a
    T[0] = t
    
    for i in range(1,N+1):
        k1 = h*f(t,w)
        k2 = h*f(t+h/2,w+k1/2)
        k3 = h*f(t+h/2,w+k2/2)
        k4 = h*f(t+h,w+k3)
        
        w = w + (k1+2*k2+2*k3+k4)/6
        y_t[i,:] = w
        t = t + h
        T[i] = t
    
    return T, y_t

In [ ]:
def May_RHS(t,w,r=3,c=0.22,alpha=505,beta=0.3,s=0.85,q=205,mu=0.03,v=0.003,e=0.031):
    N = w[0]
    P = w[1]
    dN = r*N*(1 - (c/r)*N)*((N-mu)/(v+N)) - ((alpha*N*P)/(beta+N))
    dP = s*P*(1 - ((q*P)/(N+e)))
    return np.array([dN,dP])

In [ ]:
# Limit Cycle and Unstable Equi: 

# initial conditions are slightly perturbed in N from e_5 = (N0r,P0r).

init = [N0r-0.005,P0r]
a = 0
b = 500
h = 0.001

T_limit, NP_limit = rk4(May_RHS,h,a,b,init)

N_limit = NP_limit[:,0]
P_limit = NP_limit[:,1]

N_lcyc = N_limit[-17166:] # obtains the limit cycle itself.
P_lcyc = P_limit[-17166:]
N_lcyc_trj = N_limit[:30000] # obtains a trajectory converging onto the limit cycle.
P_lcyc_trj = P_limit[:30000]

In [ ]:
# Unstable branch of e1: 

# initial conditions are slightly perturbed in P from e_1 = (r/c,0)

init = [3/0.22,0.0001]
a = 0
b = 500
h = 0.001

T_limit, NP_e1 = rk4(May_RHS,h,a,b,init)

N_e1_uns = NP_e1[:5000,0]
P_e1_uns = NP_e1[:5000,1]

In [ ]:
# stable Upper Branch of e4: 

# We have computed e_4 = (N1r,P1r) above, so initial conditions are e_4 slightly peturbed
# positively in P, to obtain upper branch. 

init = [0.03653134753893972,0.000329421207507023+0.000001]
a = 9
b = 0
h = -0.001

T_limit, NP_upper_saddle = rk4(May_RHS,h,a,b,init)

N_upper_saddle = NP_upper_saddle[:,0]
P_upper_saddle = NP_upper_saddle[:,1]

In [ ]:
# stable Lower Brance of Saddle Co-Existence equi:

# Again e_4 = (N1r,P1r) above, so initial conditions are e_4 slightly perturbed negatively
# in P, to obtain lower branch.

init = [0.03653134753893972,0.000329421207507023-0.00001]
a = 9
b = 0
h = -0.001

T_limit, NP_lower_saddle = rk4(May_RHS,h,a,b,init)

N_lower_saddle = NP_lower_saddle[:,0]
P_lower_saddle = NP_lower_saddle[:,1]

In [ ]:
# Unstable Left 

# e_4 = (N1r,P1r), initial conditions are e_4 slightly perturbed negatively in N, to obtain
# leftward branch. 

init = [0.03653134753893972-0.001,0.000329421207507023]
a = 0
b = 5
h = 0.001

T_limit, NP_left_saddle = rk4(May_RHS,h,a,b,init)

N_left_saddle = NP_left_saddle[:,0]
P_left_saddle = NP_left_saddle[:,1]

In [ ]:
# Unstable Right

# e_4 = (N1r,P1r), initial conditions are e_4 slightly perturbed positivelys in N, 
# to obtain rightward branch.

init = [0.03653134753893972+0.001,0.000329421207507023]
a = 0
b = 6
h = 0.001

T_limit, NP_right_saddle = rk4(May_RHS,h,a,b,init)

N_right_saddle = NP_right_saddle[:,0]
P_right_saddle = NP_right_saddle[:,1]

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(15,4))




ax1.hlines(0,-50,50,'k',linewidth=0.8)   # Axes
ax1.vlines(0,-50,50,'k',linewidth=0.8)
ax1.set_xlim(-0.3,14.3)
ax1.set_ylim(-0.001,0.03)

ax1.hlines(0,0,14.3,'k',linestyle='--')   # trajectory on N axis: from e_2 to e_0, from 
                                                # e_2 to e_1. 
                                          # No need to compute, dynamics worked out 
                                                # analytically in report.

ax1.plot(N_lcyc,P_lcyc,color='green',linewidth=2)   # Limit cycle
ax1.plot(N_lcyc_trj,P_lcyc_trj,'k--')               # Trajectory converging to limit cycle
ax1.plot(3.1934261414630654,0.015728908007136905,color='white',marker='o',
         markeredgecolor='r',markersize=6)   # e_5

ax1.plot(NP_e1[:8000,0],NP_e1[:8000,1],'k--')   # unstable branch of e_1
ax1.plot(3/0.22,0,'rx',markersize=8)            # e_1

ax1.plot(0.03653134753893972,0.000329421207507023,'rx',markersize=8)   # e_4
ax1.plot(N_upper_saddle,P_upper_saddle,'k--')   # All trajectories to/from e_4
ax1.plot(N_lower_saddle,P_lower_saddle,'k--')
ax1.plot(N_left_saddle,P_left_saddle,'k--')
ax1.plot(N_right_saddle,P_right_saddle,'k--')

ax1.plot(0,0,'rx',markersize=8)   # e_0
ax1.vlines(0,0,0.031/205,color='k',linestyle='--')   # trajectory on P axis: from e_0 to e_3. 
                                                     # No need to compute, dynamics worked 
                                                         # out analytically in report.

ax1.plot(0.03,0,color='white',marker='o',markeredgecolor='r',markersize=6)   # e_2

ax1.plot(0,0.031/205,'ro',markersize=6)   # e_3

ax1.set_xlabel('N', fontsize=14)
ax1.set_ylabel('P', fontsize=14, rotation=0,labelpad = 10)

ax1.text(2.8,0.0137,r'$e_5$',color='black',fontsize=12)
ax1.text(13.5,0.0014,r'$e_1$',color='black',fontsize=12)

ax1.text(2.5,0.007,r'$e_0$, $e_2$, $e_3$, $e_4$',color='black',fontsize=12)

ax1.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

ax1.text(13.6,3.05*10**(-2),r'$(a)$',color='black',fontsize=12)




# Similar for next 2 plots, just axes scaling and range changes. 





ax2.plot(N_lcyc,P_lcyc,color='green',linewidth=2)
ax2.plot(N_lcyc_trj,P_lcyc_trj,'k--')
ax2.plot(3.1934261414630654,0.015728908007136905,color='white',marker='o',
         markeredgecolor='r',markersize=6)

ax2.plot(NP_e1[:8000,0],NP_e1[:8000,1],'k--')
ax2.plot(3/0.22,0,'rx',markersize=8)

ax2.plot(0.03653134753893972,0.000329421207507023,'rx',markersize=8)
ax2.plot(N_upper_saddle,P_upper_saddle,'k--')
ax2.plot(N_lower_saddle,P_lower_saddle,'k--')
ax2.plot(N_left_saddle,P_left_saddle,'k--')
ax2.plot(N_right_saddle,P_right_saddle,'k--')

ax2.plot(0,0,'rx',markersize=8)
ax2.vlines(0,0,0.031/205,color='k',linestyle='--')

ax2.plot(0.03,0,color='white',marker='o',markeredgecolor='r',markersize=6)

ax2.plot(0,0.031/205,'ro',markersize=6)

ax2.set_yscale('log')
ax2.set_xscale('log')

ax2.set_xlim(10**(-2),10**(1.5))
ax2.set_ylim(10**(-4),10**(-1))

ax2.set_xlabel('N', fontsize=14)

ax2.text(0.03653134753893972+0.007,0.000329421207507023-0.0001,r'$e_4$',color='black',
         fontsize=12)
ax2.text(3.1934261414630654,0.015728908007136905-0.007,r'$e_5$',color='black',
         fontsize=12)

ax2.text(20.63,10**(-1)+0.01,r'$(b)$',color='black',fontsize=12)




ax3.hlines(0,-50,50,'k',linewidth=0.8)
ax3.vlines(0,-50,50,'k',linewidth=0.8)
ax3.set_xlim(-0.01,0.05)
ax3.set_ylim(-0.0001,0.0005)

ax3.hlines(0,0,14.3,'k',linestyle='--')

ax3.plot(N_lcyc,P_lcyc,color='green',linewidth=2)
ax3.plot(N_lcyc_trj,P_lcyc_trj,'k--')
ax3.plot(3.1934261414630654,0.015728908007136905,color='white',marker='o',
         markeredgecolor='r',markersize=6)

ax3.plot(NP_e1[:8000,0],NP_e1[:8000,1],'k--')
ax3.plot(3/0.22,0,'rx',markersize=8)

ax3.plot(0.03653134753893972,0.000329421207507023,'rx',markersize=8)
ax3.plot(N_upper_saddle,P_upper_saddle,'k--')
ax3.plot(N_lower_saddle,P_lower_saddle,'k--')
ax3.plot(N_left_saddle,P_left_saddle,'k--')
ax3.plot(N_right_saddle,P_right_saddle,'k--')

ax3.plot(0,0,'rx',markersize=8)
ax3.vlines(0,0,0.031/205,color='k',linestyle='--')

ax3.plot(0.03,0,color='white',marker='o',markeredgecolor='r',markersize=6)

ax3.plot(0,0.031/205,'ro',markersize=6)

ax3.set_xlabel('N', fontsize=14)

ax3.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

ax3.text(0.0012,0.1*10**(-4),r'$e_0$',color='black',fontsize=12)
ax3.text(0.0012,1.6*10**(-4),r'$e_3$',color='black',fontsize=12)
ax3.text(0.0314,0.1*10**(-4),r'$e_2$',color='black',fontsize=12)
ax3.text(0.03653134753893972+0.002,0.000329421207507023+0.2*10**(-4),r'$e_4$',color='black',
         fontsize=12)

ax3.text(0.047,5*10**(-4)+0.000008,r'$(c)$',color='black',fontsize=12)

plt.savefig('Phase Full Log Zoom.pdf',format='pdf')

### Section 5: Climate Variability

#### Figure 8:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def lorenz(x,params):
    
    x1, x2, x3 = x
    sigma, r, b = params
    
    return np.asarray([sigma*(x2 - x1), x1*(r - x3) - x2, x1*x2 - b*x3])

In [ ]:
def rk4(f, h, x0, T, alpha):
    n = int(T/h)
    t_vals = np.linspace(0, T, n)
    x = np.zeros((n, 3))
    x[0] = x0
    
    for i in range(n-1):
        k1 = h*f(x[i], alpha)
        k2 = h*f(x[i] + 0.5*k1, alpha)
        k3 = h*f(x[i] + 0.5*k2, alpha)
        k4 = h*f(x[i] + k3, alpha)
        x[i+1] = x[i] + 1/6*(k1 + 2*k2 + 2*k3 + k4)
    
    return x, t_vals

In [ ]:
sigma = 10.0
r = 28.0
b = 8.0/3.0

In [ ]:
def plot_ics(r=28.0, b=8.0/3.0, sigma=10.0, h=0.01, T=100):
    # Initialize figure and axis objects for plotting.
    fig = plt.figure(figsize=(16,10))
    ax = fig.add_subplot(projection='3d')

    # Set axis labels.
    ax.set_xlabel(r'$x(t)$',fontsize=16)
    ax.set_ylabel(r'$y(t)$',fontsize=16)
    ax.set_zlabel(r'$z(t)$',fontsize=16)

    # Disable automatic rotation of x, y and z axis labels.
    ax.xaxis.set_rotate_label(False)  
    ax.yaxis.set_rotate_label(False)  
    ax.zaxis.set_rotate_label(False)  

    x0 = np.array([1,1,10])
    x = rk4(lorenz, h, x0, T, [sigma, r, b])[0]
    ax.plot(x[:,0],x[:,1],x[:,2],'blue')
    ax.plot([0.0],[0.0],[0.0],'ko',markersize=10)
    ax.plot([(b*(r-1))**(1/2)],[(b*(r-1))**(1/2)],[r-1],'ko',markersize=10)
    ax.plot([-(b*(r-1))**(1/2)],[-(b*(r-1))**(1/2)],[r-1],'ko',markersize=10)
    
    ax.text((b*(r-1))**(1/2)+0.6,(b*(r-1))**(1/2),r-1+0.6,r'$e_1$',fontsize=15)
    ax.text(-(b*(r-1))**(1/2)+0.6,-(b*(r-1))**(1/2),r-1+0.6,r'$e_2$',fontsize=15)
    ax.text(0.6,0,0.6,r'$e_0$',fontsize=15)
    
    plt.savefig('Lorenz Trajectory.pdf',format='pdf')

In [ ]:
plot_ics(T=100)

#### Figure 9:

In [ ]:
# The following cell obtains abs_max which is our estimate of M from equation 37 
   # section 5.2 of the report. 
# M is the estimated maximum distance away that a trajectory can get from e_1 of 
   # the Lorenz system. 

def lorenz(x,params):
    
    x1, x2, x3 = x
    sigma, r, b, tau = params
    
    return np.asarray([tau*(sigma*(x2 - x1)), tau*(x1*(r - x3) - x2),
                       tau*(x1*x2 - b*x3)])

def rk4(f, h, x0, T, alpha):
    
    n = int(T/h)
    t_vals = np.linspace(0, T, n)
    x = np.zeros((n, 3))
    x[0] = x0
    
    for i in range(n-1):
        k1 = h*f(x[i], alpha)
        k2 = h*f(x[i] + 0.5*k1, alpha)
        k3 = h*f(x[i] + 0.5*k2, alpha)
        k4 = h*f(x[i] + k3, alpha)
        x[i+1] = x[i] + 1/6*(k1 + 2*k2 + 2*k3 + k4)
    
    return x, t_vals

r=28.0
b=8.0/3.0
sigma=10.0
tau=1
h=0.01
T=1000
x0 = np.array([1,1,10])

x = rk4(lorenz, h, x0, T, [sigma, r, b, tau])[0]
t_vals = rk4(lorenz, h, x0, T, [sigma, r, b, tau])[1]

e1 = np.array([np.sqrt(b*(r-1)),np.sqrt(b*(r-1)),r-1])

abs_max = max(np.linalg.norm((e1-x),axis=1))
abs_max

In [ ]:
# We set up all parameters, initial conditions and equilibrium that we need:

abs_max = 42.22323305017403

c=0.22
alpha=505
beta=0.3
s=0.85
q=205
mu=0.03
v=0.003
e=0.031

lam1 = 10.0 # sigma
lam2 = 28.0 # r
lam3 = 8.0/3.0 # b
tau = 0.4

params_may = np.array([c,alpha,beta,s,q,mu,v,e])
params_lorenz = np.array([lam1,lam2,lam3,tau])

e1 = np.array([np.sqrt(lam3*(lam2-1)),np.sqrt(lam3*(lam2-1)),lam2-1])
e2 = np.array([-np.sqrt(lam3*(lam2-1)),-np.sqrt(lam3*(lam2-1)),lam2-1])

x0_lorenz = np.array([10.74006378,  8.49421601,  7.0041354 ])
x0_may = np.array([4,1.5e-2])

d0 = np.linalg.norm(e1 - x0_lorenz)
rescaled_d0 = (1/abs_max)*d0
r0 = 2+(3-2)*rescaled_d0
r0   # This obtains the initial value of r to use, since r depends on the initial 
        # state of the Lorenz system.

In [ ]:
# We define our full system:

def system(x0,alpha,r0):
    
    N, P, x1, x2, x3 = x0
    
    c, alpha, beta, s, q, mu, v, e, lam1, lam2, lam3, tau = alpha
    
    r = r0
    
    # lam1 = sigma, lam2 = r, lam3 = b
    
    return np.asarray([r*N*(1 - (c/r)*N)*((N-mu)/(v+N)) - ((alpha*N*P)/(beta+N)),
                       s*P*(1 - ((q*P)/(N+e))),tau*(lam1*(x2 - x1)),
                       tau*(x1*(lam2 - x3) - x2), tau*(x1*x2 - lam3*x3)])


In [ ]:
# Runga kutta method that will work with our full system:

def rk4(f, h, T, x0_may, x0_lorenz, params_may, params_lorenz, r0):
    
    n = int(T/h)
    t_vals = np.linspace(0, T, n)
    x = np.zeros((n, 5))
    
    x0 = np.concatenate((x0_may,x0_lorenz))
    alpha = np.concatenate((params_may,params_lorenz))
    
    lam1, lam2, lam3, tau = params_lorenz
    e1 = np.array([np.sqrt(lam3*(lam2-1)),np.sqrt(lam3*(lam2-1)),lam2-1])
    
    x[0] = x0
    
    r0s = np.zeros((n,1))
    r0s[0] = r0
    
    for i in range(n-1):
        k1 = h*f(x[i], alpha, r0)
        k2 = h*f(x[i] + 0.5*k1, alpha, r0)
        k3 = h*f(x[i] + 0.5*k2, alpha, r0)
        k4 = h*f(x[i] + k3, alpha, r0)
        x[i+1] = x[i] + 1/6*(k1 + 2*k2 + 2*k3 + k4)
        
        d0 = np.linalg.norm(e1 - x[i+1,2:])
        rescaled_d0 = (1/abs_max)*d0
        r0 = 2+(3-2)*rescaled_d0
        
        r0s[i+1] = r0
        
    NP = x[:,:2]
    Lor = x[:,2:]
    
    return NP, Lor, t_vals, r0s

In [ ]:
# Run simulation:

NP_arr, Lor_arr, t_vals_arr, r_t = rk4(system, 0.001, 500, x0_may, x0_lorenz,
                                       params_may, params_lorenz, r0)

In [ ]:
fig = plt.figure(figsize=(10,12))



ax1 = fig.add_subplot(4, 1, (1,2), projection='3d')

points = np.array([e1,[Lor_arr[4000,0],Lor_arr[4000,1],Lor_arr[4000,2]]])

# Set axis labels.
ax1.set_xlabel(r'$x(t)$',fontsize=14,labelpad=5)
ax1.set_ylabel(r'$y(t)$',fontsize=14,labelpad=5)
ax1.set_zlabel(r'$z(t)$',fontsize=14,labelpad=0)

# Disable automatic rotation of x, y and z axis labels.
ax1.xaxis.set_rotate_label(False)  
ax1.yaxis.set_rotate_label(False)  
ax1.zaxis.set_rotate_label(False)  

ax1.set_title(r'$(a)$',loc='left',fontsize=14,pad=14)
ax1.plot(Lor_arr[1000:,0],Lor_arr[1000:,1],Lor_arr[1000:,2],'darkgray',
         linewidth=8)
ax1.plot(Lor_arr[100:4000,0],Lor_arr[100:4000,1],Lor_arr[100:4000,2],'b')
ax1.plot([Lor_arr[100,0]],[Lor_arr[100,1]],[Lor_arr[100,2]],'bo')
ax1.plot(points[:,0],points[:,1],points[:,2],'k--')
ax1.plot([Lor_arr[4000,0]],[Lor_arr[4000,1]],[Lor_arr[4000,2]],'ro')
ax1.plot([0.0],[0.0],[0.0],'ko',markersize=5)
ax1.plot([(lam3*(lam2-1))**(1/2)],[(lam3*(lam2-1))**(1/2)],[lam2-1],'ko',
         markersize=5)
ax1.plot([-(lam3*(lam2-1))**(1/2)],[-(lam3*(lam2-1))**(1/2)],[lam2-1],'ko',
         markersize=5)

box = ax1.get_position()
box.x0 = box.x0 - 0.03
box.x1 = box.x1 - 0.03
ax1.set_position(box)



ax2 = fig.add_subplot(4, 1, 3)
ax2.set_title(r'$(b)$',loc='left',fontsize=14)
ax2.plot(t_vals_arr[100:4000],r_t[100:4000],'b')
ax2.plot(t_vals_arr[100],r_t[100],'bo')
ax2.plot(t_vals_arr[4000],r_t[4000],'ro')
#ax2.vlines(t_val,2,r_val,'r')
ax2.set_xlabel('Time',fontsize=14)
ax2.set_ylabel(r'$\rho(t)$',fontsize=14,rotation=0,labelpad=13)
#ax2.set_xlim(0,10)
ax2.set_ylim(2.2,2.8)



plt.savefig('lorenz with time series - without title.pdf',format='pdf')

### Section 6: The May Model with Climate Variability

#### Figure 10:

For this figure, different initial conditions have been used for the Lorenz compared to those used in the report. This has resulted in a slightly different output, for instance the tipping for this figure occurs later than that of the tipping in the figure displayed within the report. Besides this, the code used is identical.

In [ ]:
def May_RHS(t,w,r=3,c=0.22,alpha=505,beta=0.3,s=0.85,q=205,mu=0.03,v=0.003,e=0.031):
    N = w[0]
    P = w[1]
    dN = r*N*(1 - (c/r)*N)*((N-mu)/(v+N)) - ((alpha*N*P)/(beta+N))
    dP = s*P*(1 - ((q*P)/(N+e)))
    return np.array([dN,dP])

def limcyc_rk4(f, h, a, b, init, r):
    
    N = int((b-a)/h)
    y_t = np.zeros((N+1,2))
    T = np.zeros(N+1)
    w = init
    y_t[0,:] = w
    t = a
    T[0] = t
    
    for i in range(1,N+1):
        k1 = h*f(t,w,r=r)
        k2 = h*f(t+h/2,w+k1/2,r=r)
        k3 = h*f(t+h/2,w+k2/2,r=r)
        k4 = h*f(t+h,w+k3,r=r)
        
        w = w + (k1+2*k2+2*k3+k4)/6
        y_t[i,:] = w
        t = t + h
        T[i] = t
    
    return T, y_t

T_limit, NP_limit_max = limcyc_rk4(May_RHS,0.001,0,500,np.array([4,1.5e-2]),r=3)
T_limit, NP_limit_min = limcyc_rk4(May_RHS,0.001,0,500,np.array([4,1.5e-2]),r=2)

N_limit_max = NP_limit_max[:,0]
P_limit_max = NP_limit_max[:,1]

N_limit_min = NP_limit_min[:,0]
P_limit_min = NP_limit_min[:,1]

In [ ]:
x0_lorenz = np.array([ 8.48605619, 14.33985519, 11.14232511])

abs_max = 42.22323305017403

c=0.22
alpha=505
beta=0.3
s=0.85
q=205
mu=0.03
v=0.003
e=0.031

lam1 = 10.0 # sigma
lam2 = 28.0 # r
lam3 = 8.0/3.0 # b
tau = 0.4

params_may = np.array([c,alpha,beta,s,q,mu,v,e])
params_lorenz = np.array([lam1,lam2,lam3,tau])

e1 = np.array([np.sqrt(lam3*(lam2-1)),np.sqrt(lam3*(lam2-1)),lam2-1])
e2 = np.array([-np.sqrt(lam3*(lam2-1)),-np.sqrt(lam3*(lam2-1)),lam2-1])

x0_may = np.array([4,1.5e-2])

d0 = np.linalg.norm(e1 - x0_lorenz)
rescaled_d0 = (1/abs_max)*d0
r0 = 2+(3-2)*rescaled_d0
r0

NP_arr, Lor_arr, t_vals_arr, r_t = rk4(system, 0.001, 500, x0_may, x0_lorenz, 
                                       params_may, params_lorenz, r0)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(16,5))

ax1.plot(t_vals_arr,r_t,'k')
ax1.set_xlabel('t',fontsize=14)
ax1.set_ylabel(r'$\rho(rt)$',fontsize=14,rotation=0,labelpad=13)
ax1.set_xlim(400,500)
ax1.text(400,3.05,r'$(a)$',fontsize=12)

ax2.plot(t_vals_arr,NP_arr[:,0],'k')
ax2.set_xlabel('t',fontsize=14)
ax2.set_ylabel('$N(t)$',fontsize=14,rotation=0,labelpad=12)
ax2.set_xlim(400,500)
ax2.text(400,11.1,r'$(b)$',fontsize=12)

#ax3.plot(NP_arr[200000:,0],NP_arr[200000:,1],'b',label='Population Trajectory')
ax3.plot(NP_arr[400000:500000,0],NP_arr[400000:500000,1],'k',
         label='Population Trajectory',linewidth=2)
ax3.set_yscale('log')
ax3.set_xscale('log')
ax3.set_xlim(10**(-2),13)
ax3.set_xlabel('N',fontsize=14)
ax3.set_ylabel('P',fontsize=14,rotation=0,labelpad=10)
ax3.plot(N_limit_max[-17166:],P_limit_max[-17166:],'b-',linewidth=2.5,
         label=r'limit cycle for $\rho=3$')
ax3.plot(N_limit_min[-17166:],P_limit_min[-17166:],'b--',linewidth=2.5,
         label=r'limit cycle for $\rho=2$')
ax3.legend(loc='lower right')
ax3.text(10**(-2),10**(-1.38),r'$(c)$',fontsize=12)


plt.savefig('climate variability final.pdf',format='pdf')

#### Figure 11: 

This figure was not created using code in python. I created by hand in Inkscape. 

#### Figure 12

The shading and highlighted red arc were added later in Inkscape.

In [ ]:
def May_RHS(t,w,r=3,c=0.22,alpha=505,beta=0.3,s=0.85,q=205,mu=0.03,v=0.003,e=0.031):
    N = w[0]
    P = w[1]
    dN = r*N*(1 - (c/r)*N)*((N-mu)/(v+N)) - ((alpha*N*P)/(beta+N))
    dP = s*P*(1 - ((q*P)/(N+e)))
    return np.array([dN,dP])

def limcyc_rk4(f, h, a, b, init, r):
    
    N = int((b-a)/h)
    y_t = np.zeros((N+1,2))
    T = np.zeros(N+1)
    w = init
    y_t[0,:] = w
    t = a
    T[0] = t
    
    for i in range(1,N+1):
        k1 = h*f(t,w,r=r)
        k2 = h*f(t+h/2,w+k1/2,r=r)
        k3 = h*f(t+h/2,w+k2/2,r=r)
        k4 = h*f(t+h,w+k3,r=r)
        
        w = w + (k1+2*k2+2*k3+k4)/6
        y_t[i,:] = w
        t = t + h
        T[i] = t
    
    return T, y_t

T_limit, NP_limit_max = limcyc_rk4(May_RHS,0.001,0,500,np.array([4,1.5e-2]),r=3)
T_limit, NP_limit_min = limcyc_rk4(May_RHS,0.001,0,500,np.array([4,1.5e-2]),r=2)

N_limit_max = NP_limit_max[:,0]
P_limit_max = NP_limit_max[:,1]

N_limit_min = NP_limit_min[:,0]
P_limit_min = NP_limit_min[:,1]

In [ ]:
# stable Upper Branch of e_4 r=3: 

init = [0.03653134753893972,0.000329421207507023+0.000001]
a = 9
b = 0
h = -0.001

T_limit, NP_upper_saddle = limcyc_rk4(May_RHS,h,a,b,init,r=3)

N_upper_saddle_r3 = NP_upper_saddle[:,0]
P_upper_saddle_r3 = NP_upper_saddle[:,1]

In [ ]:
# stable Upper Branch of e_4 for r=2: 

init = [0.04181182682398488,0.00035517964304382864+0.000001]
a = 12
b = 0
h = -0.001

T_limit, NP_upper_saddle = limcyc_rk4(May_RHS,h,a,b,init,r=2)

N_upper_saddle_r2 = NP_upper_saddle[:,0]
P_upper_saddle_r2 = NP_upper_saddle[:,1]

In [ ]:
plt.figure(figsize=(10,6))

plt.plot(N_limit_max[-17166:],P_limit_max[-17166:],'k-',linewidth=2.5,
         label=r'limit cycle for $\rho=3$')
plt.plot(N_limit_min[-17166:],P_limit_min[-17166:],'b-',linewidth=2.5,
         label=r'limit cycle for $\rho=2$')

plt.plot(N_upper_saddle_r3,P_upper_saddle_r3,'k--',linewidth=2.5, 
         label=r'stable manifold of saddle for $\rho=3$')

plt.plot(N_upper_saddle_r2,P_upper_saddle_r2,'b--',linewidth=2.5, 
         label=r'stable manifold of saddle for $\rho=2$')

plt.yscale('log')
plt.xscale('log')
plt.xlim(10**(-2),35)
plt.ylim(10**(-3.45),10**(-1))
plt.xlabel('N',fontsize=14)
plt.ylabel('P',fontsize=14,rotation=0,labelpad=10)

plt.legend(loc='upper left')

plt.savefig('rho = 3 and rho = 2.pdf',format='pdf')

#### Figure 13:

In [ ]:
import matplotlib.gridspec as gridspec

In [ ]:
def lorenz_tau(x,params,tau):
    
    x1, x2, x3 = x
    sigma, r, b = params
    
    return np.asarray([tau*(sigma*(x2 - x1)), tau*(x1*(r - x3) - x2), 
                       tau*(x1*x2 - b*x3)])

def system_tau(x0,alpha,r0,tau):
    
    N, P, x1, x2, x3 = x0
    
    c, alpha, beta, s, q, mu, v, e, lam1, lam2, lam3 = alpha
    
    r = r0
    
    tau_val = tau
    
    # lam1 = sigma, lam2 = r, lam3 = b
    
    return np.asarray([r*N*(1 - (c/r)*N)*((N-mu)/(v+N)) - ((alpha*N*P)/(beta+N)),
                       s*P*(1 - ((q*P)/(N+e))),tau_val*(lam1*(x2 - x1)), 
                       tau_val*(x1*(lam2 - x3) - x2), tau_val*(x1*x2 - lam3*x3)])

def rk4_tau(f, h, T, x0_may, x0_lorenz, params_may, params_lorenz, r0, rmin, tau):
    
    n = int(T/h)
    t_vals = np.linspace(0, T, n)
    x = np.zeros((n, 5))
    
    x0 = np.concatenate((x0_may,x0_lorenz))
    alpha = np.concatenate((params_may,params_lorenz))
    
    tau_val = tau 
    
    lam1, lam2, lam3 = params_lorenz
    e1 = np.array([np.sqrt(lam3*(lam2-1)),np.sqrt(lam3*(lam2-1)),lam2-1])
    e3 = np.array([0,alpha[7]/alpha[4]])
    
    x[0] = x0
    
    r0s = np.zeros((n,1))
    r0s[0] = r0
    
    for i in range(n-1):
        k1 = h*f(x[i], alpha, r0, tau_val)
        k2 = h*f(x[i] + 0.5*k1, alpha, r0, tau_val)
        k3 = h*f(x[i] + 0.5*k2, alpha, r0, tau_val)
        k4 = h*f(x[i] + k3, alpha, r0, tau_val)
        x[i+1] = x[i] + 1/6*(k1 + 2*k2 + 2*k3 + k4)
        
        d0 = np.linalg.norm(e1 - x[i+1,2:])
        rescaled_d0 = (1/abs_max)*d0
        r0 = rmin+(3-rmin)*rescaled_d0
        
        r0s[i+1] = r0
        
        if np.linalg.norm(e3 - x[i+1,:2]) <= 10**(-3):
            
            return True
        
        else:
            
            pass
        
    NP = x[:,:2]
    Lor = x[:,2:]
    
    return False

def rk4_output_tau(f, h, T, x0_may, x0_lorenz, params_may, params_lorenz, r0, rmin, tau):
    
    n = int(T/h)
    t_vals = np.linspace(0, T, n)
    x = np.zeros((n, 5))
    
    x0 = np.concatenate((x0_may,x0_lorenz))
    alpha = np.concatenate((params_may,params_lorenz))
    
    lam1, lam2, lam3 = params_lorenz
    e1 = np.array([np.sqrt(lam3*(lam2-1)),np.sqrt(lam3*(lam2-1)),lam2-1])
    e3 = np.array([0,alpha[7]/alpha[4]])
    
    x[0] = x0
    
    r0s = np.zeros((n,1))
    r0s[0] = r0
    
    for i in range(n-1):
        k1 = h*f(x[i], alpha, r0, tau)
        k2 = h*f(x[i] + 0.5*k1, alpha, r0, tau)
        k3 = h*f(x[i] + 0.5*k2, alpha, r0, tau)
        k4 = h*f(x[i] + k3, alpha, r0, tau)
        x[i+1] = x[i] + 1/6*(k1 + 2*k2 + 2*k3 + k4)
        
        d0 = np.linalg.norm(e1 - x[i+1,2:])
        rescaled_d0 = (1/abs_max)*d0
        r0 = rmin+(3-rmin)*rescaled_d0
        
        r0s[i+1] = r0
        
    NP = x[:,:2]
    Lor = x[:,2:]
    
    return NP, Lor, t_vals

In [ ]:
abs_max = 42.22323305017403

c=0.22
alpha=505
beta=0.3
s=0.85
q=205
mu=0.03
v=0.003
e=0.031

lam1 = 10.0 # sigma
lam2 = 28.0 # r
lam3 = 8.0/3.0 # b

rmin = 1.95

tau_0 = 0.1

params_may = np.array([c,alpha,beta,s,q,mu,v,e])
params_lorenz = np.array([lam1,lam2,lam3])

e1 = np.array([np.sqrt(lam3*(lam2-1)),np.sqrt(lam3*(lam2-1)),lam2-1])
e2 = np.array([-np.sqrt(lam3*(lam2-1)),-np.sqrt(lam3*(lam2-1)),lam2-1])

x0_lorenz = np.random.uniform(5,15,3)
x0_may = np.array([4,1.5e-2])

d0 = np.linalg.norm(e1 - x0_lorenz)
rescaled_d0 = (1/abs_max)*d0
r0 = 2+(3-2)*rescaled_d0
r0

<div class="alert alert-block alert-warning">
<b>Warning:</b> The following cell can take a very long time to run. 
    It is recommended that you skip the following cell and instead proceed with the  
    results it outputs which have been provided in the next cell. 
</div>

In [ ]:
tau_vals = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

probability_tau = [] 

total = 500

for tau in tau_vals:
    
    print('tau = {}'.format(tau))
    
    tipping_count = []
    
    for i in range(total):
        
        if i % 25 == 0:
            print('{}% complete.'.format((i/500)*100))
        
        x0_lorenz = np.random.uniform(5,15,3)
        
        d0 = np.linalg.norm(e1 - x0_lorenz)
        rescaled_d0 = (1/abs_max)*d0
        r0 = rmin+(3-rmin)*rescaled_d0
        r0
        
        result = rk4_tau(system_tau, 0.01, 1000, x0_may, x0_lorenz, params_may, 
                         params_lorenz, r0, rmin, tau)
        
        if result == True:
            
            tipping_count.append(1)
            
        else:
            
            pass
        
    prob = sum(tipping_count)/500.0
        
    probability_tau.append(prob)

In [ ]:
taus = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
prob_taus = np.array([0.678, 0.914, 0.884, 0.774, 0.654, 0.594,
                      0.536, 0.45, 0.4])

In [ ]:
r=1.95
c=0.22
alpha=505
beta=0.3
s=0.85
q=205
mu=0.03
v=0.003
e=0.031

A = mu - beta + (r/c) - (alpha/(c*q))
B = beta*mu + r*(beta-mu)/c - alpha*(v+e)/(c*q)
C = (r*beta*mu)/c + (alpha*v*e)/(c*q)

N0 = (2 *A**3 + 3 *np.sqrt(3)* np.sqrt(-4* A**3 *C - A**2 *B**2
               - 18 *A *B *C - 4 *B**3 + 27 *C**2 + 0j) + 9 *A *B - 
              27 *C)**(1/3)/(3 *2**(1/3)) - (2**(1/3) *(-(A)**2 - 3 *B)) / \
              (3 *(2* A**3 + 3 *np.sqrt(3) *np.sqrt(-4* A**3 *C - 
                A**2 *B**2 - 18 *A *B *C - 4* B**3 + 27 *C**2 + 0j) + 
               9* A *B - 27 *C)**(1/3)) + A/3
N1 = -((1 + np.sqrt(-1+0j)*np.sqrt(3))*(2*A**3 + 3*np.sqrt(3) * 
                np.sqrt(-4*A**3*C - A**2 * B**2 - 18 *A *B *C - 
                4 *B**3 + 27 *C**2 + 0j) + 9 *A *B - 27 *C)**(1/3)) / \
                (6 *2**(1/3)) + ((1 - np.sqrt(-1+0j) *np.sqrt(3)) * 
                (-A**2 - 3 *B))/(3 *2**(2/3) *(2 *A**3 + 3 *np.sqrt(3) * 
               np.sqrt(-4 *A**3 *C - A**2 *B**2 - 18 *A *B *C - 4 *B**3 + 
               27 *C**2 + 0j) + 9 *A *B - 27 *C)**(1/3)) + A/3
N2 = -((1 - np.sqrt(-1+0j)*np.sqrt(3))*(2*A**3 + 3*np.sqrt(3) * 
               np.sqrt(-4*A**3*C - A**2 * B**2 - 18 *A *B *C - 
               4 *B**3 + 27 *C**2 + 0j) + 9 *A *B - 27 *C)**(1/3)) / \
               (6 *2**(1/3)) + ((1 + np.sqrt(-1+0j) *np.sqrt(3)) * 
               (-A**2 - 3 *B))/(3 *2**(2/3) *(2 *A**3 + 3 *np.sqrt(3) * 
               np.sqrt(-4 *A**3 *C - A**2 *B**2 - 18 *A *B *C - 4 *B**3 + 
               27 *C**2 + 0j) + 9 *A *B - 27 *C)**(1/3)) + A/3

N0r = np.real(N0)
N1r = np.real(N1)
N2r = np.real(N2)

P0r = (N0r+e)/q 
P1r = (N1r + e)/q
P2r = (N2r+3)/q

print(N0r)
print(P0r)

print(N1r)
print(P1r)

print(N2r)
print(P2r)

IC = [N1r,P1r+0.000001]

In [ ]:
T_limit, NP_limit_min = limcyc_rk4(May_RHS,0.001,0,500,
                                   np.array([N0r+0.01,P0r+0.0001]),r=1.95)

N_limit_min = NP_limit_min[:,0]
P_limit_min = NP_limit_min[:,1]

In [ ]:
init = IC
a = 20
b = 0
h = -0.001

T_saddle, NP_saddle = limcyc_rk4(May_RHS,h,a,b,init,r=1.95)

N_saddle = NP_saddle[:,0]
P_saddle = NP_saddle[:,1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8,8), 
                               gridspec_kw={'height_ratios': [1, 2]})


for i, prob in enumerate(prob_taus):
    ax1.vlines(taus[i],0,prob,color='k',linestyle='--',linewidth=1.5)
ax1.scatter(taus,prob_taus,color='r',marker='o')
ax1.set_ylim(0,1.05)
ax1.set_xlabel('r',fontsize=14)
ax1.set_ylabel('Pr',fontsize=14,rotation=90,labelpad=3)

ax1.text(0.07,1.07,r'$(a)$',fontsize=12)


ax2.plot(N_limit_max[-17166:],P_limit_max[-17166:],'k-',linewidth=2.5,
         label=r'limit cycle for $\rho=3$')
ax2.plot(N_limit_min[-17166:],P_limit_min[-17166:],'b-',linewidth=2.5,
         label=r'limit cycle for $\rho=1.95$')

ax2.plot(N_saddle,P_saddle,'b--',linewidth=2.5, label=r'$W^s(e_4)$ for $\rho=1.95$')

ax2.set_yscale('log')
ax2.set_xscale('log')

ax2.set_xlim(10**(-1.8),30)
ax2.set_ylim(10**(-3.45),10**(-1))

ax2.set_xlabel('N',fontsize=14)
ax2.set_ylabel('P',fontsize=14,rotation=0,labelpad=10)

ax2.text(10**(-2)+0.007,10**(-1)+0.01,r'$(b)$',fontsize=12)

ax2.legend(loc='upper left')

plt.savefig('ProbTip vs r - without title.pdf',format='pdf')

#### Figure 14:

In [ ]:
def lorenz(x,params):
    
    x1, x2, x3 = x
    sigma, r, b, tau = params
    
    return np.asarray([tau*(sigma*(x2 - x1)), tau*(x1*(r - x3) - x2), 
                       tau*(x1*x2 - b*x3)])

def system(x0,alpha,r0):
    
    N, P, x1, x2, x3 = x0
    
    c, alpha, beta, s, q, mu, v, e, lam1, lam2, lam3, tau = alpha
    
    r = r0
    
    # lam1 = sigma, lam2 = r, lam3 = b
    
    return np.asarray([r*N*(1 - (c/r)*N)*((N-mu)/(v+N)) - ((alpha*N*P)/(beta+N)),
                       s*P*(1 - ((q*P)/(N+e))),tau*(lam1*(x2 - x1)), 
                       tau*(x1*(lam2 - x3) - x2), tau*(x1*x2 - lam3*x3)])

def rk4(f, h, T, x0_may, x0_lorenz, params_may, params_lorenz, r0, rmin):
    
    n = int(T/h)
    t_vals = np.linspace(0, T, n)
    x = np.zeros((n, 5))
    
    x0 = np.concatenate((x0_may,x0_lorenz))
    alpha = np.concatenate((params_may,params_lorenz))
    
    lam1, lam2, lam3, tau = params_lorenz
    e1 = np.array([np.sqrt(lam3*(lam2-1)),np.sqrt(lam3*(lam2-1)),lam2-1])
    e3 = np.array([0,alpha[7]/alpha[4]])
    
    x[0] = x0
    
    r0s = np.zeros((n,1))
    r0s[0] = r0
    
    for i in range(n-1):
        k1 = h*f(x[i], alpha, r0)
        k2 = h*f(x[i] + 0.5*k1, alpha, r0)
        k3 = h*f(x[i] + 0.5*k2, alpha, r0)
        k4 = h*f(x[i] + k3, alpha, r0)
        x[i+1] = x[i] + 1/6*(k1 + 2*k2 + 2*k3 + k4)
        
        d0 = np.linalg.norm(e1 - x[i+1,2:])
        rescaled_d0 = (1/abs_max)*d0
        r0 = rmin+(3-rmin)*rescaled_d0
        
        r0s[i+1] = r0
        
        if np.linalg.norm(e3 - x[i+1,:2]) <= 10**(-3):
            
            return True
        
        else:
            
            pass
        
    NP = x[:,:2]
    Lor = x[:,2:]
    
    return False

def rk4_output(f, h, T, x0_may, x0_lorenz, params_may, params_lorenz, r0, rmin):
    
    n = int(T/h)
    t_vals = np.linspace(0, T, n)
    x = np.zeros((n, 5))
    
    x0 = np.concatenate((x0_may,x0_lorenz))
    alpha = np.concatenate((params_may,params_lorenz))
    
    lam1, lam2, lam3, tau = params_lorenz
    e1 = np.array([np.sqrt(lam3*(lam2-1)),np.sqrt(lam3*(lam2-1)),lam2-1])
    e3 = np.array([0,alpha[7]/alpha[4]])
    
    x[0] = x0
    
    r0s = np.zeros((n,1))
    r0s[0] = r0
    
    for i in range(n-1):
        k1 = h*f(x[i], alpha, r0)
        k2 = h*f(x[i] + 0.5*k1, alpha, r0)
        k3 = h*f(x[i] + 0.5*k2, alpha, r0)
        k4 = h*f(x[i] + k3, alpha, r0)
        x[i+1] = x[i] + 1/6*(k1 + 2*k2 + 2*k3 + k4)
        
        d0 = np.linalg.norm(e1 - x[i+1,2:])
        rescaled_d0 = (1/abs_max)*d0
        r0 = rmin+(3-rmin)*rescaled_d0
        
        r0s[i+1] = r0
        
    NP = x[:,:2]
    Lor = x[:,2:]
    
    return NP, Lor, t_vals

In [ ]:
abs_max = 42.22323305017403

c=0.22
alpha=505
beta=0.3
s=0.85
q=205
mu=0.03
v=0.003
e=0.031

lam1 = 10.0 # sigma
lam2 = 28.0 # r
lam3 = 8.0/3.0 # b
tau = 0.4

rmin_0 = 1.8

params_may = np.array([c,alpha,beta,s,q,mu,v,e])
params_lorenz = np.array([lam1,lam2,lam3,tau])

e1 = np.array([np.sqrt(lam3*(lam2-1)),np.sqrt(lam3*(lam2-1)),lam2-1])
e2 = np.array([-np.sqrt(lam3*(lam2-1)),-np.sqrt(lam3*(lam2-1)),lam2-1])

x0_lorenz = np.random.uniform(5,15,3)
x0_may = np.array([4,1.5e-2])

d0 = np.linalg.norm(e1 - x0_lorenz)
rescaled_d0 = (1/abs_max)*d0
r0 = rmin_0+(3-rmin_0)*rescaled_d0
r0

<div class="alert alert-block alert-warning">
<b>Warning:</b> The following cell can take a very long time to run. 
    It is recommended that you skip the following cell and instead proceed with the  
    results it outputs which have been provided in the next cell. 
</div>

In [ ]:
# We completed two runs of this code: 
# r1_min = [2.0,2.2,2.4,2.6,2.8,3.0]  =>  everything past and 
                         # including 2.2 had probability zero.
# r2_min = [1.8,1.85,1.9,1.95,2.0,2.05,2.1,2.15,2.2]  =>  First 
                         # zero probability for 2.15. 

# We thus infer that if we ran:

# r_min = [1.8,1.85,1.9,1.95,2.0,2.05,2.1,2.15,2.2,2.25,
            # 2.3,2.35,2.4,2.45,2.5,2.55,2.6,2.65,2.7,
            # 2.75,2.8,2.85,2.9,2.95,3.0]

# We would have zeroes for every value past and including 2.15:

r_min = [1.8,1.85,1.9,1.95,2.0,2.05,2.1,2.15,2.2]

probability = [] 

total = 500

for rmin in r_min:
    
    print('rmin = {}'.format(rmin))
    
    tipping_count = []
    
    for i in range(total):
        
        if i % 25 == 0:
            print('{}% complete.'.format((i/500)*100))
        
        x0_lorenz = np.random.uniform(5,15,3)
        
        d0 = np.linalg.norm(e1 - x0_lorenz)
        rescaled_d0 = (1/abs_max)*d0
        r0 = rmin+(3-rmin)*rescaled_d0
        r0
        
        result = rk4(system, 0.01, 1000, x0_may, x0_lorenz, params_may,
                     params_lorenz, r0, rmin)
        
        if result == True:
            
            tipping_count.append(1)
            
        else:
            
            pass
        
    prob = sum(tipping_count)/500.0
        
    probability.append(prob)

In [ ]:
rmins = np.array([1.8,1.85,1.9,1.95,2.0,2.05,2.1,2.15,2.2,
                  2.25,2.3,2.35,2.4,2.45,2.5,2.55,2.6,2.65,
                  2.7,2.75,2.8,2.85,2.9,2.95,3.0])
probs = np.array([1, 0.992, 0.946, 0.796, 0.476, 0.188, 0.048,
                  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

In [ ]:
r1=1.8
r2=2.298
r3=2.5
rlist = [r1,r2,r3]
c=0.22
alpha=505
beta=0.3
s=0.85
q=205
mu=0.03
v=0.003
e=0.031

Nlist_limcyc = []
Plist_limcyc = []

Nlist_IC = []
Plist_IC = []

for r in rlist:   
    
    A = mu - beta + (r/c) - (alpha/(c*q))
    B = beta*mu + r*(beta-mu)/c - alpha*(v+e)/(c*q)
    C = (r*beta*mu)/c + (alpha*v*e)/(c*q)

    N0 = (2 *A**3 + 3 *np.sqrt(3)* np.sqrt(-4* A**3 *C - A**2 *B**2
                   - 18 *A *B *C - 4 *B**3 + 27 *C**2 + 0j) + 9 *A *B - 
                  27 *C)**(1/3)/(3 *2**(1/3)) - (2**(1/3) *(-(A)**2 - 3 *B)) / \
                  (3 *(2* A**3 + 3 *np.sqrt(3) *np.sqrt(-4* A**3 *C - 
                    A**2 *B**2 - 18 *A *B *C - 4* B**3 + 27 *C**2 + 0j) + 
                   9* A *B - 27 *C)**(1/3)) + A/3
    N1 = -((1 + np.sqrt(-1+0j)*np.sqrt(3))*(2*A**3 + 3*np.sqrt(3) * 
                    np.sqrt(-4*A**3*C - A**2 * B**2 - 18 *A *B *C - 
                    4 *B**3 + 27 *C**2 + 0j) + 9 *A *B - 27 *C)**(1/3)) / \
                    (6 *2**(1/3)) + ((1 - np.sqrt(-1+0j) *np.sqrt(3)) * 
                    (-A**2 - 3 *B))/(3 *2**(2/3) *(2 *A**3 + 3 *np.sqrt(3) * 
                   np.sqrt(-4 *A**3 *C - A**2 *B**2 - 18 *A *B *C - 4 *B**3 + 
                   27 *C**2 + 0j) + 9 *A *B - 27 *C)**(1/3)) + A/3
    N2 = -((1 - np.sqrt(-1+0j)*np.sqrt(3))*(2*A**3 + 3*np.sqrt(3) * 
                   np.sqrt(-4*A**3*C - A**2 * B**2 - 18 *A *B *C - 
                   4 *B**3 + 27 *C**2 + 0j) + 9 *A *B - 27 *C)**(1/3)) / \
                   (6 *2**(1/3)) + ((1 + np.sqrt(-1+0j) *np.sqrt(3)) * 
                   (-A**2 - 3 *B))/(3 *2**(2/3) *(2 *A**3 + 3 *np.sqrt(3) * 
                   np.sqrt(-4 *A**3 *C - A**2 *B**2 - 18 *A *B *C - 4 *B**3 + 
                   27 *C**2 + 0j) + 9 *A *B - 27 *C)**(1/3)) + A/3

    N0r = np.real(N0)
    N1r = np.real(N1)
    N2r = np.real(N2)

    P0r = (N0r+e)/q 
    P1r = (N1r + e)/q
    P2r = (N2r+3)/q
    
    Nlist_limcyc.append(N0r)
    Nlist_IC.append(N1r)
    
    Plist_limcyc.append(P0r)
    Plist_IC.append(P1r)
    
Narr_limcyc = np.array(Nlist_limcyc)
Parr_limcyc = np.array(Plist_limcyc)

Narr_IC = np.array(Nlist_IC)
Parr_IC = np.array(Plist_IC)

In [ ]:
T_limit, NP_limit_min_r1 = limcyc_rk4(May_RHS,0.001,0,500,
                          np.array([Narr_limcyc[0]+0.01,Parr_limcyc[0]+0.0001]),r1)
T_limit, NP_limit_min_r2 = limcyc_rk4(May_RHS,0.001,0,500,
                          np.array([Narr_limcyc[1]+0.01,Parr_limcyc[1]+0.0001]),r2)
T_limit, NP_limit_min_r3 = limcyc_rk4(May_RHS,0.001,0,500,
                          np.array([Narr_limcyc[2]+0.01,Parr_limcyc[2]+0.0001]),r3)

N_limit_min_r1 = NP_limit_min_r1[:,0]
P_limit_min_r1 = NP_limit_min_r1[:,1]

N_limit_min_r2 = NP_limit_min_r2[:,0]
P_limit_min_r2 = NP_limit_min_r2[:,1]

N_limit_min_r3 = NP_limit_min_r3[:,0]
P_limit_min_r3 = NP_limit_min_r3[:,1]

In [ ]:
init1 = [Narr_IC[0],Parr_IC[0]+0.000001]
init2 = [Narr_IC[1],Parr_IC[1]+0.000001]
init3 = [Narr_IC[2],Parr_IC[2]+0.000001]

a = 20
b = 0
h = -0.001

T_saddle, NP_saddle_1 = limcyc_rk4(May_RHS,h,a,b,init1,r1)
T_saddle, NP_saddle_2 = limcyc_rk4(May_RHS,h,a,b,init2,r2)
T_saddle, NP_saddle_3 = limcyc_rk4(May_RHS,h,a,b,init3,r3)

N_saddle_r1 = NP_saddle_1[:,0]
P_saddle_r1 = NP_saddle_1[:,1]

N_saddle_r2 = NP_saddle_2[:,0]
P_saddle_r2 = NP_saddle_2[:,1]

N_saddle_r3 = NP_saddle_3[:,0]
P_saddle_r3 = NP_saddle_3[:,1]

In [ ]:
# Plot figure with subplots of different sizes
fig = plt.figure(figsize=(14,10))
# set up subplot grid
gridspec.GridSpec(2,3)



# large subplot
plt.subplot2grid((2,3), (0,0), colspan=3, rowspan=1)

plt.plot(rmins,probs,'k',linewidth=2.5)

plt.vlines(2.298,0,1,'k',linestyle='--',linewidth=1.5,
           label='Basin Instability Threshold')

plt.xlim(1.8,2.5)

plt.xlabel(r'Minimum $\rho$',fontsize=14,labelpad=-1)
plt.ylabel('Pr',fontsize=14,rotation=90,labelpad=3)

plt.text(1.805,1.07,r'$(a)$',fontsize=12)

plt.legend(loc='upper right')



# small subplot 1
plt.subplot2grid((2,3), (1,0))

plt.plot(N_limit_max[-17166:],P_limit_max[-17166:],'k-',
         linewidth=2.5,label=r'$\rho=3$')
plt.plot(N_limit_min_r1[-17166:],P_limit_min_r1[-17166:],'b-',
         linewidth=2.5,label=r'$\rho=1.8$')

plt.plot(N_saddle_r1,P_saddle_r1,'b',linestyle='--',linewidth=2.5,
         label=r'$W^s(e_4)$ for $\rho=1.8$')

plt.yscale('log')
plt.xscale('log')

plt.xlim(10**(-1.8),30)
plt.ylim(10**(-3.45),10**(-1))

plt.xlabel('N',fontsize=14)
plt.ylabel('P',fontsize=14,rotation=0,labelpad=10)

plt.text(10**(-2)+0.01,10**(-1)+0.01,r'$(b)$',fontsize=12)

plt.legend(loc='upper left')




# small subplot 2
plt.subplot2grid((2,3), (1,1))

plt.plot(N_limit_max[-17166:],P_limit_max[-17166:],'k-',
         linewidth=2.5,label=r'$\rho=3$')
plt.plot(N_limit_min_r2[-17166:],P_limit_min_r2[-17166:],'b-',
         linewidth=2.5,label=r'$\rho=2.298$')

plt.plot(N_saddle_r2,P_saddle_r2,'b--',linewidth=2.5, 
         label=r'$W^s(e_4)$ for $\rho=2.298$')

plt.yscale('log')
plt.xscale('log')

plt.xlim(10**(-1.8),30)
plt.ylim(10**(-3.45),10**(-1))

plt.xlabel('N',fontsize=14)

plt.text(10**(-2)+0.01,10**(-1)+0.01,r'$(c)$',fontsize=12)

plt.legend()



# small subplot 3
plt.subplot2grid((2,3), (1,2))

plt.plot(N_limit_max[-17166:],P_limit_max[-17166:],'k-',
         linewidth=2.5,label=r'$\rho=3$')
plt.plot(N_limit_min_r3[-17166:],P_limit_min_r3[-17166:],'b-',
         linewidth=2.5,label=r'$\rho=2.5$')

plt.plot(N_saddle_r3,P_saddle_r3,'b--',linewidth=2.5, label=r'$W^s(e_4)$ for $\rho=2.5$')

plt.yscale('log')
plt.xscale('log')

plt.xlim(10**(-1.8),30)
plt.ylim(10**(-3.45),10**(-1))

plt.xlabel('N',fontsize=14)

plt.text(10**(-2)+0.01,10**(-1)+0.01,r'$(d)$',fontsize=12)

plt.legend()

plt.savefig('ProbTip vs rho min - without title.pdf',format='pdf')